# Check GPU

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Dec 12 17:05:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Run this command once and restart the runtime

In [4]:
!pip install av

     |████████████████████████████████| 36.9MB 169kB/s 


# HMDB51 + DataLoader, Spatial

In [5]:
import glob
import os

from torchvision.datasets.utils import list_dir
from torchvision.datasets.folder import make_dataset
from torchvision.datasets.video_utils import VideoClips
from torchvision.datasets.vision import VisionDataset

class HMDB51(VisionDataset):
    """
    Internally, it uses a VideoClips object to handle clip creation.
    Args:
        root (string): Root directory of the HMDB51 Dataset.
        frames_per_clip (int): Number of frames in a clip.
        step_between_clips (int): Number of frames between each clip.
        train (bool, optional): If ``True``, creates a dataset from the train split,
            otherwise from the ``test`` split.
        transform (callable, optional): A function/transform that takes in a TxHxWxC video
            and returns a transformed version.
    Returns:
        video (Tensor[T, H, W, C]): the `T` video frames
        audio(Tensor[K, L]): the audio frames, where `K` is the number of channels
            and `L` is the number of points
        label (int): class of the video clip
    """
    def __init__(self, root=""):
        super(HMDB51, self).__init__(root)

    def init_data(self, root, frames_per_clip, step_between_clips=1,
                 frame_rate=None, train=True, transform=None,
                 _precomputed_metadata=None, num_workers=1, _video_width=0,
                 _video_height=0, _video_min_dimension=0, _audio_samples=0):
        super(HMDB51, self).__init__(root)
        extensions = ('avi',)
        if train:
            root = root + "/train"
        else:
            root = root + "/test"
        classes = sorted(list_dir(root))
        class_to_idx = {class_: i for (i, class_) in enumerate(classes)}
        print(class_to_idx)
        self.samples = []
        for target_class in sorted(class_to_idx.keys()):
            class_index = class_to_idx[target_class]
            target_dir = os.path.join(root, target_class)
            for root_curr, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root_curr, fname)
                    if os.path.isfile(path):
                        item = path, class_index
                        self.samples.append(item)

        video_paths = [path for (path, _) in self.samples]
        video_clips = VideoClips(
            video_paths,
            frames_per_clip,
            step_between_clips,
            frame_rate,
            _precomputed_metadata,
            num_workers=num_workers,
            _video_width=_video_width,
            _video_height=_video_height,
            _video_min_dimension=_video_min_dimension,
            _audio_samples=_audio_samples,
        )
        self.train = train
        self.classes = classes
        self.video_clips_metadata = video_clips.metadata
        self.indices = self.get_indices(video_paths)
        self.video_clips = video_clips.subset(self.indices)
        self.transform = transform

    @property
    def metadata(self):
        return self.video_clips_metadata

    def get_indices(self, video_list):
        indices = []
        for video_index, video_path in enumerate(video_list):
            indices.append(video_index)
        return indices

    def __len__(self):
        return self.video_clips.num_clips()

    def __getitem__(self, idx):
        video, _, _, video_idx = self.video_clips.get_clip(idx)
        sample_index = self.indices[video_idx]
        _, class_index = self.samples[sample_index]

        if self.transform is not None:
            video = video.permute(0, 3, 1, 2)
            video = self.transform(video)
            video = video.permute(0, 2, 3, 1)

        return video, class_index, sample_index

    def state_dict(self):
        state = {"video_clips": self.video_clips,
                 "indices": self.indices,
                 "samples": self.samples,
                 "transform": self.transform,
                 "metadata": self.video_clips_metadata}

        return state

    def load_state_dict(self, state):
        self.video_clips = state["video_clips"]
        self.indices = state["indices"]
        self.samples = state["samples"]
        self.transform = state["transform"]
        self.video_clips_metadata = state["metadata"]

import torch
from torch.utils.data import DataLoader

class BuildDataLoader(torch.utils.data.DataLoader):
    def __init__(self, dataset, batch_size, shuffle, num_workers):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_workers = num_workers
        self.w_frame = 224
        self.h_frame = 224

    def collect_fn(self, batch):
        video_list = []
        label_list = []
        index_list = []

        for video, cl_index, s_index in batch:
            video_list.append(video)
            label_list.append(cl_index)
            index_list.append(s_index)

        data = {"videos": torch.stack(video_list),
                "labels": label_list,
                "indexes": index_list
                }

        return data

    def loader(self):
        return DataLoader(self.dataset,
                          batch_size=self.batch_size,
                          shuffle=self.shuffle,
                          num_workers=self.num_workers,
                          collate_fn=self.collect_fn)

# Mount drive


In [7]:
import os
import io
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/drive'

drive.mount(DRIVE_MOUNT)


# create folder to write data to
DATA_FOLDER = os.path.join(DRIVE_MOUNT, 'Shared drives', 'CIS680 Final Project', 'data')

Mounted at /content/drive


# Load Dataset and Dataloaders

In [9]:
import matplotlib.pyplot as plt



train_dataset = HMDB51(DATA_FOLDER)
test_dataset = HMDB51(DATA_FOLDER)

train_dataset.load_state_dict(torch.load(DATA_FOLDER + "/train_dataset_3.pt"))
test_dataset.load_state_dict(torch.load(DATA_FOLDER + "/test_dataset_3.pt"))

batch_size = 64 # reduced batch size for fuse since Cuda runs out of memory for 128
train_build_loader = BuildDataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
train_loader = train_build_loader.loader()
test_build_loader = BuildDataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = test_build_loader.loader()

print("Length of train_loader: {}".format(len(train_loader)))
print("Length of test_loader: {}".format(len(test_loader)))

Length of train_loader: 7536
Length of test_loader: 2208


# Spatial Model

In [11]:
import torch
from torch.nn import functional as F
from torchvision import transforms
from torch import nn, Tensor
import random
import torchvision
import torchvision.models as models
import torch.optim as optim


class SpatialStream(torch.nn.Module):

    def __init__(self, 
                 device='cuda',
                 num_classes=51,
                 dropout_probability=0.5,
                 train_resnet=True):

        # Initialize the stream layers
        super(SpatialStream, self).__init__()
        self.device = device
        self.num_classes = num_classes

        # Spatial Pretrained Resnet-50
        self.spatial = models.resnet50(pretrained=True)
        for param in self.spatial.parameters():
            param.requires_grad = train_resnet  # False: Freezes the weights of the pre-trained model

        # Add FC layers to Spatial Resnet-50
        self.spatial.fc = nn.Sequential(nn.Linear(2048, 1024),
                                nn.ReLU(),
                                nn.Dropout(p=dropout_probability),
                                nn.Linear(1024, self.num_classes),
                                nn.ReLU(),
                                nn.Dropout(p=dropout_probability),
                                nn.Softmax())

    def forward(self, X):
        return self.spatial(X)

    def compute_loss(self, output, labels):
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, labels)
        return loss

# Temporal Model

In [12]:
class TemporalStream(torch.nn.Module):

    def __init__(self,
                 device='cuda',
                 num_classes=51,
                 dropout_probability=0.5):

        # Initialize the stream layers
        super(TemporalStream, self).__init__()
        self.device = device
        self.num_classes = num_classes

        # Temporal Pretrained Resnet-50
        self.temporal = models.resnet50(pretrained=True)
        for param in self.temporal.parameters():
            param.requires_grad = True

        # Add FC layers to Temporal Resnet-50
        self.temporal.fc = nn.Sequential(nn.Linear(2048, 1024),
                                nn.ReLU(),
                                nn.Dropout(p=dropout_probability),
                                nn.Linear(1024, self.num_classes),
                                nn.ReLU(),
                                nn.Dropout(p=dropout_probability),
                                nn.Softmax())
        

    def forward(self, X):
        X = self.temporal(X)
        return X

    def compute_loss(self, output, target):
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, target)
        return loss

# Fuse Model

In [13]:
class FuseNET(torch.nn.Module):

    def __init__(self, 
                 device='cuda',
                 num_classes=51):

        # Initialize the FuseNet FC layers
        super(FuseNET, self).__init__()
        self.device = device
        self.num_classes = num_classes

        self.fc1 = nn.Linear(102, 204)
        self.output = nn.Linear(204, self.num_classes)

    def forward(self, X):
        X = self.fc1(X)
        X = self.output(X)
        return X

    def compute_loss(self, output, labels):
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, labels)
        return loss

# Train & Test Functions

In [14]:
from google.colab.patches import cv2_imshow

def getSG3I(videos):
    bz = videos.size(0)

    frame_list_batch = []

    for b in range(bz):
        images = videos[b]

        w = images[0].size(0)
        h = images[0].size(1)
        num_frames = images.size(0)
        frame_list = []

        for i in range(0, num_frames):
            frame_list.append(cv.cvtColor(images[i].numpy(), cv.COLOR_BGR2GRAY))

        frame_list_batch.append(np.stack(frame_list, axis=-1))

    SG3I = np.stack(frame_list_batch, axis=0)

    return torch.Tensor(SG3I)

In [35]:
  def train(epoch):
    spatial.eval()
    temporal.eval()
    fused.train()

    counter = 0
    train_loss = 0
    log_interval = 100
    save_interval = 250

    epoch_loss = []
    log_int_loss = 0
    for iter, data in enumerate(train_loader, 7499):
        print("iter = ", iter)
        if iter <= 7499:
          continue
        if iter > 7536:
          break

        videos = data["videos"]
        labels = torch.tensor(data["labels"])
        indexes = data["indexes"]
        
        SG3I = getSG3I(videos)
        SG3I = SG3I.permute(0,3,1,2)
        SG3I = SG3I.to(device)

        videos = videos.type(torch.FloatTensor)
        videos = videos.to(device)
        # labels = labels.to(device)

        optimizer.zero_grad()

        # spatial
        spatial_input = videos[:,0,:,:].permute(0,3,2,1)
        spatial_output = spatial(spatial_input)

        # temporal        
        temporal_output = temporal(SG3I)
        
        # fused
        fused_input = torch.hstack((spatial_output, temporal_output))
        fused_output = fused(fused_input)

        fused_output = fused_output.to(device)
        labels = labels.to(device)

        # calculate losses
        loss = fused.compute_loss(fused_output, labels)

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Logging Interval
        log_int_loss += loss.item()
        epoch_loss.append(loss.item())

        if counter == 0:
            print('Epoch: ', epoch, ', Batch: ', iter, ', loss avg over log interval: ', log_int_loss)
            train_loss_list.append(train_loss / (iter + 1) * batch_size)
            train_counter.append((iter + 1) * batch_size + epoch * len(train_loader.dataset))
            log_int_loss = 0
        elif counter % log_interval == log_interval - 1:
            print('Epoch: ', epoch, ', Batch: ', iter, ', loss avg over log interval: ', log_int_loss / log_interval)
            train_loss_list.append(train_loss / (iter + 1) * batch_size)
            train_counter.append((iter + 1) * batch_size + epoch * len(train_loader.dataset))
            log_int_loss = 0

        if counter % save_interval == save_interval - 1:
            print('saving model')
            save_path = os.path.join(EPOCH_SAVE_PREFIX, 'fused_epoch' + str(epoch) + '_iter_' + str(counter))
            torch.save({
                'epoch': epoch,
                'train_total_loss_list': train_loss_list,
                'epoch_total_loss_list': epoch_loss_list,
                'test_loss_list': test_loss_list,
                'train_counter': train_counter,
                'accuracy_list': accuracy_list,
                'model_state_dict': fused.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, save_path)

        counter += 1


    avg_loss = sum(epoch_loss) / len(epoch_loss)
    epoch_loss_list.append(avg_loss)
    print('Epoch: ', epoch, ', avg total loss: ', avg_loss)

def test():
    fused.eval()
    test_loss = 0
    correct = 0

    # Data Loop
    with torch.no_grad():
        for iter, data in enumerate(test_loader, 0):
            videos = data["videos"]
            labels = torch.tensor(data["labels"])
            indexes = data["indexes"]
            
            SG3I = getSG3I(videos)
            SG3I = SG3I.permute(0,3,1,2)
            SG3I = SG3I.to(device)

            videos = videos.type(torch.FloatTensor)
            videos = videos.to(device)
            # labels = labels.to(device)

            # spatial
            spatial_input = videos[:,0,:,:].permute(0,3,2,1)
            spatial_output = spatial(spatial_input)

            # temporal        
            temporal_output = temporal(SG3I)
            
            # fused
            fused_input = torch.hstack((spatial_output, temporal_output))
            fused_output = fused(fused_input)

            fused_output = fused_output.to(device)
            labels = labels.to(device)

            # calculate losses
            loss = fused.compute_loss(fused_output, labels)

            test_loss += loss.item()

            # calculate number of correct predictions in batch
            correct += sum(torch.argmax(fused_output,1) == labels).item()
            if iter % 100 == 0:
                print ("iter  ", iter)
                print("accuracy so far = ", correct / ((iter + 1) * len(labels)))

    # Log
    test_loss_list.append(test_loss / len(test_loader.dataset))
    accuracy = correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print('Avg Validation Loss: ', test_loss / len(test_loader.dataset))
    print('Accuracy: ', accuracy)

# Main Training Loop

In [ ]:
EPOCH_SAVE_PREFIX = '/content/drive/Shared drives/CIS680 Final Project/models/fused/'

# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.random.manual_seed(1)

# load trained spatial
spatial = SpatialStream()
spatial.to(device)
spatial_network_path = '/content/drive/Shared drives/CIS680 Final Project/models/spatial/spatial_epoch10'
checkpoint_spatial = torch.load(spatial_network_path)
spatial.load_state_dict(checkpoint_spatial['model_state_dict'])

# load trained temporal
temporal = TemporalStream()
temporal.to(device)
temporal_network_path = '/content/drive/Shared drives/CIS680 Final Project/models/temporal_SG3I/temporal_epoch8'
checkpoint_temporal = torch.load(temporal_network_path)
temporal.load_state_dict(checkpoint_temporal['model_state_dict'])


# initialize fused model
learning_rate = 0.001
fused = FuseNET()
fused.to(device)
optimizer = optim.SGD(fused.parameters(), lr=learning_rate, momentum=0.9)

# Epochs
num_epochs = 50

# Logging setup: train
train_loss_list = []
epoch_loss_list = []
train_counter = []

# Logging setup: test
test_loss_list = []
accuracy_list = []
epoch_list = np.arange(num_epochs)

# epoch loop
for epoch in range(num_epochs):

    # Train & Validate
    train(epoch)
    test()

    # Save Model Version
    save_path = os.path.join(EPOCH_SAVE_PREFIX, 'fused_epoch' + str(epoch))
    torch.save({
        'epoch': epoch,
        'train_total_loss_list': train_loss_list,
        'epoch_total_loss_list': epoch_loss_list,
        'test_loss_list': test_loss_list,
        'train_counter': train_counter,
        'accuracy_list': accuracy_list,
        'model_state_dict': fused.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, save_path)

    print("Epoch %d/%d Completed" % (epoch, num_epochs - 1))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:117: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch:  0 , Batch:  0 , loss avg over log interval:  3.9206459522247314
Epoch:  0 , Batch:  99 , loss avg over log interval:  3.8703181767463684
Epoch:  0 , Batch:  199 , loss avg over log interval:  3.872889759540558
saving model
Epoch:  0 , Batch:  299 , loss avg over log interval:  3.8393946290016174
Epoch:  0 , Batch:  399 , loss avg over log interval:  3.8074916434288024
Epoch:  0 , Batch:  499 , loss avg over log interval:  3.7731434655189515
saving model
Epoch:  0 , Batch:  599 , loss avg over log interval:  3.7234731125831604
Epoch:  0 , Batch:  699 , loss avg over log interval:  3.695622055530548
saving model
Epoch:  0 , Batch:  799 , loss avg over log interval:  3.6621727991104125
Epoch:  0 , Batch:  899 , loss avg over log interval:  3.634461441040039
Epoch:  0 , Batch:  999 , loss avg over log interval:  3.5999487257003784
saving model
Epoch:  0 , Batch:  1099 , loss avg over log interval:  3.5693886423110963
Epoch:  0 , Batch:  1199 , loss avg over log interval:  3.5482602

# Resume Training

In [ ]:
EPOCH_SAVE_PREFIX = '/content/drive/Shared drives/CIS680 Final Project/models/fused/'

# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.random.manual_seed(1)

# load trained spatial
spatial = SpatialStream()
spatial.to(device)
spatial_network_path = '/content/drive/Shared drives/CIS680 Final Project/models/spatial/spatial_epoch10'
checkpoint_spatial = torch.load(spatial_network_path)
spatial.load_state_dict(checkpoint_spatial['model_state_dict'])

# load trained temporal
temporal = TemporalStream()
temporal.to(device)
temporal_network_path = '/content/drive/Shared drives/CIS680 Final Project/models/temporal_SG3I/temporal_epoch8'
checkpoint_temporal = torch.load(temporal_network_path)
temporal.load_state_dict(checkpoint_temporal['model_state_dict'])


# FuseNet model
learning_rate = 0.001
fused = FuseNET()
fused.to(device)
optimizer = optim.SGD(fused.parameters(), lr=learning_rate, momentum=0.9)

# Epochs
num_epochs = 50
epoch_list = np.arange(num_epochs)


# Load FuseNet model
fused_network_path = '/content/drive/Shared drives/CIS680 Final Project/models/fused/fused_epoch1_iter_7499'
checkpoint_fused = torch.load(fused_network_path)
fused.load_state_dict(checkpoint_fused['model_state_dict'])
optimizer.load_state_dict(checkpoint_fused['optimizer_state_dict'])
last_epoch = checkpoint_fused['epoch']

# Logging setup
train_loss_list = checkpoint_fused['train_total_loss_list']
epoch_loss_list = checkpoint_fused['epoch_total_loss_list']
test_loss_list = checkpoint_fused['test_loss_list']
accuracy_list = checkpoint_fused['accuracy_list']
train_counter = checkpoint_fused['train_counter']


# epoch loop
for epoch in range(1, num_epochs):

    # Train & Validate
    train(epoch)
    test()

    # Save Model Version
    save_path = os.path.join(EPOCH_SAVE_PREFIX, 'fused_epoch' + str(epoch))
    torch.save({
        'epoch': epoch,
        'train_total_loss_list': train_loss_list,
        'epoch_total_loss_list': epoch_loss_list,
        'test_loss_list': test_loss_list,
        'train_counter': train_counter,
        'accuracy_list': accuracy_list,
        'model_state_dict': fused.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, save_path)

    print("Epoch %d/%d Completed" % (epoch, num_epochs - 1))

/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:117: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."


iter =  7499
iter =  7500


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch:  1 , Batch:  7500 , loss avg over log interval:  1.4026941061019897
iter =  7501
iter =  7502
iter =  7503
iter =  7504
iter =  7505
iter =  7506
iter =  7507
iter =  7508
iter =  7509
iter =  7510
iter =  7511
iter =  7512
iter =  7513
iter =  7514
iter =  7515
iter =  7516
iter =  7517
iter =  7518
iter =  7519
iter =  7520
iter =  7521
iter =  7522
iter =  7523
iter =  7524
iter =  7525
iter =  7526
iter =  7527
iter =  7528
iter =  7529
iter =  7530
iter =  7531
iter =  7532
iter =  7533
iter =  7534
iter =  7535
iter =  7536
iter =  7537
Epoch:  1 , avg total loss:  1.398154616355896
iter   0
accuracy so far =  0.921875
iter   100
accuracy so far =  0.5323329207920792
iter   200
accuracy so far =  0.42319651741293535
iter   300
accuracy so far =  0.5083056478405316
iter   400
accuracy so far =  0.49423316708229426
iter   500
accuracy so far =  0.4937624750499002


# Calculate Accuracy of a Saved Model

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model
learning_rate = 0.001
spatial = SpatialStream()
spatial.to(device)
optimizer = optim.SGD(spatial.parameters(), lr=learning_rate, momentum=0.9)

# LOAD NETWORK
network_path = '/content/drive/Shared drives/CIS680 Final Project/models/spatial/spatial_epoch0'
checkpoint = torch.load(network_path)
spatial.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']

# Logging setup: train
train_loss_list = checkpoint['train_total_loss_list']
epoch_loss_list = checkpoint['epoch_total_loss_list']
test_loss_list = checkpoint['test_loss_list']
train_counter = checkpoint['train_counter']

test()